## implementação dos algoritmos de recorte

In [9]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import xml.etree.ElementTree as ET
import math

class Visualizador:
    def __init__(self, root):
        self.root = root
        self.root.title("Visualizador de Objetos 2D")
    
        # Margem de clipping
        self.clipping_margin = 30
        
        # DisplayList window reference
        self.displaylist_window = None
    
        # Configurar o menu
        menu = tk.Menu(root)
        root.config(menu=menu)
        file_menu = tk.Menu(menu, tearoff=False)
        menu.add_cascade(label="Arquivo", menu=file_menu)
        file_menu.add_command(label="Abrir", command=self.abrir_arquivo)
        file_menu.add_command(label="Gerar XML de Saída", command=self.gerar_xml_saida)

        # Adiciona o menu Sobre
        sobre_menu = tk.Menu(menu, tearoff=False)
        menu.add_cascade(label="Sobre", menu=sobre_menu)
        sobre_menu.add_command(label="DisplayList", command=self.mostrar_displaylist)
        sobre_menu.add_command(label="Controles do Teclado", command=self.mostrar_controles_do_teclado)
    
        # Frame principal para conter canvas e painel lateral
        frame_principal = tk.Frame(root)
        frame_principal.pack(side="top", fill="both", expand=True)
    
        # canvas da Viewport principal
        self.canvas = tk.Canvas(frame_principal, width=800, height=600, bg="white")
        self.canvas.pack(side="left", fill="both", expand=True)
    
        # painel lateral
        right_panel = tk.Frame(frame_principal, width=180)
        right_panel.pack(side="right", fill="y", padx=10, pady=10)
        right_panel.pack_propagate(False)

        # Frame container centralizado
        center_container = tk.Frame(right_panel)
        center_container.place(relx=0.5, rely=0.5, anchor="center")

        # canvas do minimapa
        self.minimap = tk.Canvas(center_container, width=150, height=120, bg="lightgrey", relief='sunken', bd=1)
        self.minimap.pack(side="top", pady=(0,10))

        # Variável para o algoritmo de clipping
        self.algoritmo_clipping_var = tk.StringVar(value="cohen-sutherland")

        # Caixa de seleção
        frame_recorte_linhas = tk.LabelFrame(center_container, text="Recorte (Linhas)", padx=6, pady=6, relief='groove')
        frame_recorte_linhas.pack(side="top", fill="x", pady=(0, 10))

        rb_cohen = tk.Radiobutton(
            frame_recorte_linhas,
            text="Cohen-Sutherland",
            variable=self.algoritmo_clipping_var,
            value="cohen-sutherland",
            indicatoron=1,
            command=lambda: self._atualizar_algoritmo(self.algoritmo_clipping_var.get())
        )
        rb_cohen.pack(anchor="w", pady=2)

        rb_liang = tk.Radiobutton(
            frame_recorte_linhas,
            text="Liang-Barsky",
            variable=self.algoritmo_clipping_var,
            value="liang-barsky",
            indicatoron=1,
            command=lambda: self._atualizar_algoritmo(self.algoritmo_clipping_var.get())
        )
        rb_liang.pack(anchor="w", pady=2)

        # Caixa de seleção "Recorte (Poligonos)"
        frame_recorte_poligonos = tk.LabelFrame(center_container, text="Recorte (Polígonos)", padx=6, pady=6, relief='groove')
        frame_recorte_poligonos.pack(side="top", fill="x", pady=(0, 10))

        btn_weiler = tk.Button(frame_recorte_poligonos, text="→ Weiler-Atherton", anchor='w', command=self._dummy_polygons_button)
        btn_weiler.pack(fill="x")

        # frame para os botões de navegação
        frame_botoes = tk.Frame(root)
        frame_botoes.pack(side="bottom", anchor="w", padx=10, pady=10)

        # botões de navegação
        btn_up = tk.Button(frame_botoes, text="↑", width=3, command=lambda: self._mover_e_recarregar(0, 1))
        btn_down = tk.Button(frame_botoes, text="↓", width=3, command=lambda: self._mover_e_recarregar(0, -1))
        btn_left = tk.Button(frame_botoes, text="←", width=3, command=lambda: self._mover_e_recarregar(-1, 0))
        btn_right = tk.Button(frame_botoes, text="→", width=3, command=lambda: self._mover_e_recarregar(1, 0))

        btn_up.grid(row=0, column=1, padx=3, pady=3)
        btn_left.grid(row=1, column=0, padx=3, pady=3)
        btn_right.grid(row=1, column=2, padx=3, pady=3)
        btn_down.grid(row=2, column=1, padx=3, pady=3)

        # botões de Zoom
        btn_zoom_in = tk.Button(frame_botoes, text="+", width=3, command=lambda: self._zoom(0.9)) 
        btn_zoom_out = tk.Button(frame_botoes, text="-", width=3, command=lambda: self._zoom(1.1))
        btn_zoom_in.grid(row=1, column=3, padx=(15, 3))
        btn_zoom_out.grid(row=1, column=4, padx=3)

        # botões de Rotação
        btn_rot_left = tk.Button(frame_botoes, text="⟲", width=3, command=lambda: self._rotacionar(-2.0))
        btn_rot_right = tk.Button(frame_botoes, text="⟳", width=3, command=lambda: self._rotacionar(2.0))
        btn_rot_left.grid(row=1, column=5, padx=(15, 3))
        btn_rot_right.grid(row=1, column=6, padx=3)

        # Dados
        self.objects = []
        self.window = {'wmin':(0.0,0.0), 'wmax':(1.0,1.0)}
        self.viewport = {'vpmin':(0.0,0.0), 'vpmax':(800.0,600.0)}
        self.rotation_angle = 0.0
        self.M = [[1.0,0.0,0.0],[0.0,1.0,0.0],[0.0,0.0,1.0]]
        self.world_bounds = {'xmin': 0.0, 'xmax': 1.0, 'ymin': 0.0, 'ymax': 1.0}
        self.caminho_arquivo = None

        # Bind das teclas
        root.bind("<Left>", lambda e: self._mover_e_recarregar(-1,0))
        root.bind("<Right>", lambda e: self._mover_e_recarregar(1,0))
        root.bind("<Up>", lambda e: self._mover_e_recarregar(0,1))
        root.bind("<Down>", lambda e: self._mover_e_recarregar(0,-1))
        root.bind("<plus>", lambda e: self._zoom(0.9))
        root.bind("<equal>", lambda e: self._zoom(0.9))
        root.bind("<KP_Add>", lambda e: self._zoom(0.9))
        root.bind("<minus>", lambda e: self._zoom(1.1))
        root.bind("<KP_Subtract>", lambda e: self._zoom(1.1))
        root.bind("<comma>", lambda e: self._rotacionar(-2.0))
        root.bind("<period>", lambda e: self._rotacionar(2.0))
    
    def _dummy_polygons_button(self):
        messagebox.showinfo("Recorte (Polígonos)", "Usando Weiler-Atherton.")

    def _atualizar_algoritmo(self, novo_algoritmo):
        self.algoritmo_clipping_var.set(novo_algoritmo)
        if self.objects:
            self.desenhar_viewport()
            self.atualizar_displaylist()
    
    def _rotacionar(self, angulo):
        self.rotation_angle += angulo
        
        # Normaliza o ângulo entre 0 e 360 graus
        self.rotation_angle = self.rotation_angle % 360
        
        # Se for negativo, converte para o equivalente positivo
        if self.rotation_angle < 0:
            self.rotation_angle += 360
        self.recalcular_matriz()
        self.desenhar_viewport()
        self.desenhar_minimapa()
        self.atualizar_displaylist()

    def abrir_arquivo(self):
        caminho = filedialog.askopenfilename(filetypes=[("XML files","*.xml"),("All files","*.*")])
        if not caminho:
            return
        try:
            self.caminho_arquivo = caminho
            self.carregar_arquivo(caminho)
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao carregar arquivo:\n{e}")

    def carregar_arquivo(self, caminho):
        tree = ET.parse(caminho)
        root = tree.getroot()

        # parse viewport
        vp = root.find('viewport')
        if vp is not None:
            vpmin = vp.find('vpmin')
            vpmax = vp.find('vpmax')
            if vpmin is not None and vpmax is not None:
                try:
                    self.viewport['vpmin'] = (float(vpmin.get('x', '0')), float(vpmin.get('y', '0')))
                    self.viewport['vpmax'] = (float(vpmax.get('x', '800')), float(vpmax.get('y', '600')))
                except ValueError:
                    pass

        vpxmin, vpymin = self.viewport['vpmin']
        vpxmax, vpymax = self.viewport['vpmax']
        largura = max(200, int(abs(vpxmax - vpxmin)))
        altura = max(200, int(abs(vpymax - vpymin)))
        self.canvas.config(width=largura, height=altura)

        # parse window
        w = root.find('window')
        if w is not None:
            wmin = w.find('wmin')
            wmax = w.find('wmax')
            if wmin is not None and wmax is not None:
                try:
                    self.window['wmin'] = (float(wmin.get('x','0')), float(wmin.get('y','0')))
                    self.window['wmax'] = (float(wmax.get('x','1')), float(wmax.get('y','1')))
                except ValueError:
                    pass

        # parse objetos
        self.objects = []
        for child in root:
            tag = child.tag.lower() if child.tag else ''
            if child.get('coords') == 'viewport':
                continue
            if tag == 'viewport' or tag == 'window':
                continue
            color = child.get('cor', 'black')
            
            if tag == 'ponto':
                try:
                    x = float(child.get('x','0'))
                    y = float(child.get('y','0'))
                    self.objects.append({
                        'type':'ponto', 
                        'points':[(x,y)], 
                        'color': color,
                        'visivel': False,
                        'points_ppc': [],
                        'clipped_points': []
                    })
                except ValueError:
                    continue
            elif tag == 'reta':
                pts = []
                for p in child.findall('ponto'):
                    try:
                        x = float(p.get('x','0'))
                        y = float(p.get('y','0'))
                        pts.append((x, y))
                    except ValueError:
                        continue
                if len(pts) >= 2:
                    self.objects.append({
                        'type':'reta', 
                        'points':pts[:2], 
                        'color': color,
                        'visivel': False,
                        'points_ppc': [],
                        'clipped_points': []
                    })
            elif tag == 'poligono':
                pts = []
                for p in child.findall('ponto'):
                    try:
                        x = float(p.get('x','0'))
                        y = float(p.get('y','0'))
                        pts.append((x, y))
                    except ValueError:
                        continue
                if len(pts) >= 3:
                    self.objects.append({
                        'type':'poligono', 
                        'points':pts, 
                        'color': color,
                        'visivel': False,
                        'points_ppc': [],
                        'clipped_polygons': []
                    })

        self.calcular_limites_mundo()
        self.window['wmin'] = (self.world_bounds['xmin'], self.world_bounds['ymin'])
        self.window['wmax'] = (self.world_bounds['xmax'], self.world_bounds['ymax'])
        self.rotation_angle = 0.0

        self.recalcular_matriz()
        self.desenhar_viewport()
        self.desenhar_minimapa()
        self.atualizar_displaylist()
        
    def calcular_limites_mundo(self):
        if not self.objects:
            wxmin, wymin = self.window['wmin']
            wxmax, wymax = self.window['wmax']
            self.world_bounds = {'xmin': wxmin, 'xmax': wxmax, 'ymin': wymin, 'ymax': wymax}
            return
        
        xs = []
        ys = []
        for obj in self.objects:
            for x, y in obj['points']:
                xs.append(x)
                ys.append(y)
                
        if xs and ys:
            xmin, xmax = min(xs), max(xs)
            ymin, ymax = min(ys), max(ys)
            wwidth = self.window['wmax'][0] - self.window['wmin'][0]
            wheight = self.window['wmax'][1] - self.window['wmin'][1]
            margin_x = abs(wwidth) * 0.3 if wwidth != 0 else 1.0
            margin_y = abs(wheight) * 0.3 if wheight != 0 else 1.0
            self.world_bounds = {
                'xmin': xmin - margin_x,
                'xmax': xmax + margin_x,
                'ymin': ymin - margin_y,
                'ymax': ymax + margin_y
            }

    def recalcular_matriz(self):
        wxmin, wymin = self.window['wmin']
        wxmax, wymax = self.window['wmax']
        vpxmin, vpymin = self.viewport['vpmin']
        vpxmax, vpymax = self.viewport['vpmax']
        if (wxmax - wxmin) == 0: sx = 1.0
        else: sx = (vpxmax - vpxmin) / (wxmax - wxmin)
        if (wymax - wymin) == 0: sy = 1.0
        else: sy = (vpymax - vpymin) / (wymax - wymin)
        tx = vpxmin - sx * wxmin
        ty = vpymin + sy * wymax
        self.M = [[sx, 0.0, tx], [0.0, -sy, ty], [0.0, 0.0, 1.0]]

    # Transforma ponto do mundo para PPC
    def world_to_ppc(self, ponto):
        xw, yw = ponto
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']

        window_center_x = (wminx + wmaxx) / 2
        window_center_y = (wminy + wmaxy) / 2

        # Translada para origem
        translated_x = xw - window_center_x
        translated_y = yw - window_center_y

        # Rotaciona (ângulo inverso)
        angle_rad = math.radians(-self.rotation_angle)
        cos_a = math.cos(angle_rad)
        sin_a = math.sin(angle_rad)
        rotated_x = translated_x * cos_a - translated_y * sin_a
        rotated_y = translated_x * sin_a + translated_y * cos_a
        return (rotated_x, rotated_y)

    # Transforma ponto do PPC para viewport
    def ppc_to_viewport(self, ponto_ppc):
        xppc, yppc = ponto_ppc
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']
        
        largura = int(self.canvas.cget('width'))
        altura = int(self.canvas.cget('height'))
        
        # Viewport
        vpxmin = self.clipping_margin
        vpymin = self.clipping_margin
        vpxmax = largura - self.clipping_margin
        vpymax = altura - self.clipping_margin

        window_width = wmaxx - wminx
        window_height = wmaxy - wminy
        viewport_width = vpxmax - vpxmin
        viewport_height = vpymax - vpymin

        if window_width == 0 or window_height == 0: 
            return (0, 0)
        
        norm_x = xppc / (window_width / 2)
        norm_y = yppc / (window_height / 2)
        vp_x = (norm_x * (viewport_width / 2)) + (viewport_width / 2) + vpxmin
        vp_y = (-norm_y * (viewport_height / 2)) + (viewport_height / 2) + vpymin
        return (vp_x, vp_y)

    # Clipping de ponto no PPC
    def clip_ponto(self, ponto_ppc):
        x, y = ponto_ppc
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']
        
        w = wmaxx - wminx
        h = wmaxy - wminy
        
        if -w/2 <= x <= w/2 and -h/2 <= y <= h/2:
            return True, ponto_ppc
        return False, None

    # Calcula a regiao para Cohen-Sutherland
    def calcular_region_code(self, x, y, xmin, ymin, xmax, ymax):
        INSIDE = 0  # 0000
        LEFT = 1    # 0001
        RIGHT = 2   # 0010
        BOTTOM = 4  # 0100
        TOP = 8     # 1000
        
        code = INSIDE
        if x < xmin:
            code |= LEFT
        elif x > xmax:
            code |= RIGHT
        if y < ymin:
            code |= BOTTOM
        elif y > ymax:
            code |= TOP
        return code
    
    # Clipping de reta no PPC
    def clip_reta_cohen_sutherland(self, p1_ppc, p2_ppc):
        x1, y1 = p1_ppc
        x2, y2 = p2_ppc
        
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']
        w = wmaxx - wminx
        h = wmaxy - wminy
        
        # Limites da window no PPC
        xmin, xmax = -w/2, w/2
        ymin, ymax = -h/2, h/2
        
        code1 = self.calcular_region_code(x1, y1, xmin, ymin, xmax, ymax)
        code2 = self.calcular_region_code(x2, y2, xmin, ymin, xmax, ymax)
        aceito = False
        
        # Algoritmo iterativo
        while True:
            # Aceitação trivial ambos dentro
            if (code1 | code2) == 0:
                aceito = True
                break
            # Rejeição trivial ambos na mesma região externa
            elif (code1 & code2) != 0:
                break
            else:
                # Escolhe um ponto fora
                if code1 != 0:
                    code_out = code1
                else:
                    code_out = code2
                # Calcula coeficiente angular
                if (x2 - x1) != 0:
                    m = (y2 - y1) / (x2 - x1)
                else:
                    m = float('inf')  # Reta vertical
                # Calcula intersecção conforme equação: y - y1 = m(x - x1)
                x, y = 0, 0
                if code_out & 8:  # TOP (bit 1000)
                    y = ymax
                    if m != 0 and m != float('inf'):
                        x = x1 + (ymax - y1) / m
                    else:
                        x = x1     
                elif code_out & 4:  # BOTTOM (bit 0100)
                    y = ymin
                    if m != 0 and m != float('inf'):
                        x = x1 + (ymin - y1) / m
                    else:
                        x = x1      
                elif code_out & 2:  # RIGHT (bit 0010)
                    x = xmax
                    if m != float('inf'):
                        y = y1 + m * (xmax - x1)
                    else:
                        y = y1      
                elif code_out & 1:  # LEFT (bit 0001)
                    x = xmin
                    if m != float('inf'):
                        y = y1 + m * (xmin - x1)
                    else:
                        y = y1
                # Atualiza ponto e recalcula código
                if code_out == code1:
                    x1, y1 = x, y
                    code1 = self.calcular_region_code(x1, y1, xmin, ymin, xmax, ymax)
                else:
                    x2, y2 = x, y
                    code2 = self.calcular_region_code(x2, y2, xmin, ymin, xmax, ymax)
        if aceito:
            return True, [(x1, y1), (x2, y2)]
        return False, None

    # Clipping de reta no PPC
    def clip_reta_liang_barsky(self, p1_ppc, p2_ppc):
        x1, y1 = p1_ppc
        x2, y2 = p2_ppc
        
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']
        w = wmaxx - wminx
        h = wmaxy - wminy
        
        xmin, xmax = -w/2, w/2
        ymin, ymax = -h/2, h/2
        
        dx = x2 - x1
        dy = y2 - y1
        
        u1 = 0.0
        u2 = 1.0
        
        # Define p e q conforme especificação
        p = [-dx, dx, -dy, dy]
        q = [x1 - xmin, xmax - x1, y1 - ymin, ymax - y1]
        
        # Testa cada borda
        for i in range(4):
            if p[i] == 0:
                # Linha paralela à borda
                if q[i] < 0:
                    return False, None
            else:
                r = q[i] / p[i]
                if p[i] < 0:
                    # Entrada de fora para dentro
                    u1 = max(u1, r)
                else:
                    # Saída de dentro para fora
                    u2 = min(u2, r)
        
        # Verifica se há segmento visível
        if u1 > u2:
            return False, None
        
        # Calcula novos pontos
        x1_new = x1 + u1 * dx
        y1_new = y1 + u1 * dy
        x2_new = x1 + u2 * dx
        y2_new = y1 + u2 * dy
        
        return True, [(x1_new, y1_new), (x2_new, y2_new)]

    # Clipping de polígono no PPC
    def clip_poligono_weiler_atherton(self, pontos_ppc):
        if len(pontos_ppc) < 3:
            return []

        # Define os vértices do retângulo de clipping no PPC
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']
        w = wmaxx - wminx
        h = wmaxy - wminy
        xmin, xmax = -w/2,  w/2
        ymin, ymax = -h/2,  h/2
        clip_vertices = [(xmin, ymin), (xmax, ymin), (xmax, ymax), (xmin, ymax)]

        EPS = 1e-10
        def inside(p):
            x, y = p
            return (x >= xmin - EPS) and (x <= xmax + EPS) and (y >= ymin - EPS) and (y <= ymax + EPS)

        # Interseção entre segmentos
        def seg_intersection(p1, p2, q1, q2):
            x1, y1 = p1
            x2, y2 = p2
            x3, y3 = q1
            x4, y4 = q2
            den = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
            if abs(den) < EPS:
                return None
            t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / den
            u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / den
            if -EPS <= t <= 1 + EPS and -EPS <= u <= 1 + EPS:
                xi = x1 + t * (x2 - x1)
                yi = y1 + t * (y2 - y1)
                return (xi, yi, max(0.0, min(1.0, t)), max(0.0, min(1.0, u)))
            return None

        class Node:
            __slots__ = ("x","y","isect","enter","twin","next","prev","visited","owner")
            def __init__(self, x, y, owner, isect=False, enter=None):
                self.x = x
                self.y = y
                self.owner = owner
                self.isect = isect
                self.enter = enter
                self.twin = None
                self.next = None
                self.prev = None
                self.visited = False

            def pt(self):
                return (self.x, self.y)

        def make_circular_nodes(pts, owner):
            nodes = [Node(px, py, owner, isect=False, enter=None) for (px, py) in pts]
            n = len(nodes)
            for i in range(n):
                nodes[i].next = nodes[(i+1) % n]
                nodes[(i+1) % n].prev = nodes[i]
            return nodes

        subj_nodes = make_circular_nodes(pontos_ppc, "subj")
        clip_nodes = make_circular_nodes(clip_vertices, "clip")

        isects = []
        subj_to_idx = {node: i for i, node in enumerate(subj_nodes)}
        clip_to_idx = {node: i for i, node in enumerate(clip_nodes)}

        for i, s in enumerate(subj_nodes):
            s_next = s.next
            p1, p2 = s.pt(), s_next.pt()
            for j, c in enumerate(clip_nodes):
                c_next = c.next
                q1, q2 = c.pt(), c_next.pt()
                res = seg_intersection(p1, p2, q1, q2)
                if not res:
                    continue
                xi, yi, t_s, t_c = res

                # Calcula o vetor da aresta do polígono e o vetor normal da borda do clip
                vx, vy = (p2[0]-p1[0]), (p2[1]-p1[1])
                nx, ny = (q2[1]-q1[1]), (q1[0]-q2[0])
                prod = vx*nx + vy*ny
                entering = prod < 0

                isects.append({
                    "point": (xi, yi),
                    "subj_edge": i,
                    "clip_edge": j,
                    "t_subj": t_s,
                    "t_clip": t_c,
                    "enter": entering
                })

        if not isects:
            if all(inside(n.pt()) for n in subj_nodes):
                return [pontos_ppc]
            return []

        from collections import defaultdict
        subj_bucket = defaultdict(list)
        clip_bucket = defaultdict(list)
        for rec in isects:
            subj_bucket[rec["subj_edge"]].append(rec)
            clip_bucket[rec["clip_edge"]].append(rec)
        for k in subj_bucket:
            subj_bucket[k].sort(key=lambda r: r["t_subj"])
        for k in clip_bucket:
            clip_bucket[k].sort(key=lambda r: r["t_clip"])

        for i, s in enumerate(subj_nodes):
            inserts = subj_bucket.get(i, [])
            anchor = s
            for rec in inserts:
                xi, yi = rec["point"]
                nd = Node(xi, yi, "subj", isect=True, enter=rec["enter"])
                nxt = anchor.next
                anchor.next = nd
                nd.prev = anchor
                nd.next = nxt
                nxt.prev = nd
                anchor = nd
                rec["node_subj"] = nd

        for j, c in enumerate(clip_nodes):
            inserts = clip_bucket.get(j, [])
            anchor = c
            for rec in inserts:
                xi, yi = rec["point"]
                nd = Node(xi, yi, "clip", isect=True, enter=not rec["enter"])
                nxt = anchor.next
                anchor.next = nd
                nd.prev = anchor
                nd.next = nxt
                nxt.prev = nd
                anchor = nd
                rec["node_clip"] = nd

        for rec in isects:
            a = rec.get("node_subj")
            b = rec.get("node_clip")
            if a and b:
                a.twin = b
                b.twin = a
        result = []

        def iter_list(head):
            cur = head
            first = True
            while first or (cur is not head):
                first = False
                yield cur
                cur = cur.next

        subj_all = []
        for head in subj_nodes:
            for node in iter_list(head):
                if node.isect:
                    subj_all.append(node)
            break

        for start in subj_all:
            if not start.isect or not start.enter or start.visited:
                continue
            contour = []
            cur = start
            cur_list = "subj"
            while True:
                contour.append((cur.x, cur.y))
                if cur.isect:
                    cur.visited = True
                    if cur.twin:
                        cur.twin.visited = True
                nxt = cur.next
                cur = nxt
                if cur.isect:
                    if cur.twin is None:
                        break
                    cur = cur.twin
                    cur_list = "clip" if cur_list == "subj" else "subj"
                if cur is start:
                    contour.append((cur.x, cur.y))
                    break
                if len(contour) > 5000:
                    break
            if contour and len(contour) >= 4:
                if abs(contour[0][0]-contour[-1][0]) < 1e-8 and abs(contour[0][1]-contour[-1][1]) < 1e-8:
                    contour = contour[:-1]
            if len(contour) >= 3:
                result.append(contour)
        if not result and all(inside(n.pt()) for n in subj_nodes):
            return [pontos_ppc]

        # Remove possíveis duplicatas numéricas muito próximas
        def dedup_ring(ring):
            cleaned = []
            for pt in ring:
                if not cleaned or (abs(pt[0]-cleaned[-1][0]) > 1e-8 or abs(pt[1]-cleaned[-1][1]) > 1e-8):
                    cleaned.append(pt)
            if len(cleaned) >= 2 and abs(cleaned[0][0]-cleaned[-1][0]) < 1e-8 and abs(cleaned[0][1]-cleaned[-1][1]) < 1e-8:
                cleaned.pop()
            return cleaned

        result = [dedup_ring(r) for r in result if len(dedup_ring(r)) >= 3]
        return result

    # Realiza clipping de todos os objetos
    def realizar_clipping(self):
        for obj in self.objects:
            # Transforma para PPC
            obj['points_ppc'] = [self.world_to_ppc(p) for p in obj['points']]
            
            if obj['type'] == 'ponto':
                visivel, ponto_clip = self.clip_ponto(obj['points_ppc'][0])
                obj['visivel'] = visivel
                obj['clipped_points'] = [ponto_clip] if visivel else []
                
            elif obj['type'] == 'reta':
                p1_ppc, p2_ppc = obj['points_ppc'][0], obj['points_ppc'][1]
                
                # Escolhe algoritmo
                if self.algoritmo_clipping_var.get() == "cohen-sutherland":
                    visivel, pontos_clip = self.clip_reta_cohen_sutherland(p1_ppc, p2_ppc)
                else:
                    visivel, pontos_clip = self.clip_reta_liang_barsky(p1_ppc, p2_ppc)
                
                obj['visivel'] = visivel
                obj['clipped_points'] = pontos_clip if visivel else []
                
            elif obj['type'] == 'poligono':
                poligonos_clip = self.clip_poligono_weiler_atherton(obj['points_ppc'])
                obj['visivel'] = len(poligonos_clip) > 0
                obj['clipped_polygons'] = poligonos_clip

    def desenhar_viewport(self):
        self.canvas.delete("all")
        
        largura = int(self.canvas.cget('width'))
        altura = int(self.canvas.cget('height'))
        
        # Desenha o fundo cinza primeiro
        self.canvas.create_rectangle(0, 0, largura, altura, fill='lightgray', outline='')
        
        # realiza o clipping para ter os dados atualizados
        self.realizar_clipping()

        # lógica para definir a cor da borda 
        borda_sendo_tocada = False
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']
        w = wmaxx - wminx
        h = wmaxy - wminy
        xmin_ppc, xmax_ppc = -w/2, w/2
        ymin_ppc, ymax_ppc = -h/2, h/2

        for obj in self.objects:
            if obj['visivel']:
                for x_ppc, y_ppc in obj['points_ppc']:
                    if not (xmin_ppc <= x_ppc <= xmax_ppc and ymin_ppc <= y_ppc <= ymax_ppc):
                        borda_sendo_tocada = True
                        break
            if borda_sendo_tocada:
                break
        
        cor_da_borda = "red" if borda_sendo_tocada else "black"

        # desenha a área branca interna
        self.canvas.create_rectangle(
            self.clipping_margin, 
            self.clipping_margin, 
            largura - self.clipping_margin, 
            altura - self.clipping_margin, 
            fill='white', 
            outline="" 
        )
        
        # desenha TODOS os objetos recortados
        for obj in self.objects:
            if not obj['visivel']:
                continue
            color = obj['color']

            if obj['type'] == 'ponto':
                ponto_ppc = obj['clipped_points'][0]
                cx, cy = self.ppc_to_viewport(ponto_ppc)
                r = 3
                self.canvas.create_oval(cx-r, cy-r, cx+r, cy+r, fill=color)

            elif obj['type'] == 'reta':
                p1_ppc, p2_ppc = obj['clipped_points']
                p1_vp = self.ppc_to_viewport(p1_ppc)
                p2_vp = self.ppc_to_viewport(p2_ppc)
                self.canvas.create_line(p1_vp[0], p1_vp[1], p2_vp[0], p2_vp[1], width=2, fill=color)

            elif obj['type'] == 'poligono':
                for poligono_ppc in obj['clipped_polygons']:
                    pts_vp = [self.ppc_to_viewport(p) for p in poligono_ppc]
                    flat_pts = [coord for point in pts_vp for coord in point]
                    if len(flat_pts) >= 6:
                        self.canvas.create_line(*flat_pts, flat_pts[0], flat_pts[1], width=2, fill=color)

        # desenha a borda por cima de tudo
        self.canvas.create_rectangle(
            self.clipping_margin, 
            self.clipping_margin, 
            largura - self.clipping_margin, 
            altura - self.clipping_margin, 
            fill='', 
            outline=cor_da_borda, 
            width=2
        )

        # Desenha o texto de informações
        algoritmo_atual = "Cohen-Sutherland" if self.algoritmo_clipping_var.get() == "cohen-sutherland" else "Liang-Barsky"
        wxmin, wymin = self.window['wmin']
        wxmax, wmaxy = self.window['wmax']
        window_center_x = (wxmin + wxmax) / 2
        window_center_y = (wymin + wmaxy) / 2
        
        txt = (f"Algoritmo: {algoritmo_atual}\n"
            f"Ângulo: {self.rotation_angle:.2f}°\n"
            f"Window Center: ({window_center_x:.2f}, {window_center_y:.2f})")
            
        self.canvas.create_text(
            self.clipping_margin + 8, 
            self.clipping_margin + 8, 
            anchor='nw', 
            text=txt, 
            font=("Courier", 9), 
            fill='blue'
        )

    # desenhar a viewport do minimapa
    def desenhar_minimapa(self):
        self.minimap.delete("all")
        xs, ys = [], []
        for obj in self.objects:
            for x,y in obj['points']:
                xs.append(x); ys.append(y)
        xs.extend([self.window['wmin'][0], self.window['wmax'][0], self.world_bounds['xmin'], self.world_bounds['xmax']])
        ys.extend([self.window['wmin'][1], self.window['wmax'][1], self.world_bounds['ymin'], self.world_bounds['ymax']])
        if not xs: return
        xmin, xmax = min(xs), max(xs)
        ymin, ymax = min(ys), max(ys)
        if xmin == xmax: xmin -= 1; xmax += 1
        if ymin == ymax: ymin -= 1; ymax += 1
        pad_x = (xmax - xmin) * 0.08
        pad_y = (ymax - ymin) * 0.08
        xmin -= pad_x; xmax += pad_x; ymin -= pad_y; ymax += pad_y
        mw = int(self.minimap.cget('width'))
        mh = int(self.minimap.cget('height'))

        # Converte ponto do mundo para minimapa
        def world_to_minimap(px, py):
            range_x = xmax - xmin
            range_y = ymax - ymin
            if range_x == 0 or range_y == 0:
                return 0, 0
            sx = mw / range_x
            sy = mh / range_y
            mx = (px - xmin) * sx
            my = mh - (py - ymin) * sy
            return mx, my

        # Desenha limites do mundo
        p1 = world_to_minimap(self.world_bounds['xmin'], self.world_bounds['ymin'])
        p2 = world_to_minimap(self.world_bounds['xmin'], self.world_bounds['ymax'])
        p3 = world_to_minimap(self.world_bounds['xmax'], self.world_bounds['ymax'])
        p4 = world_to_minimap(self.world_bounds['xmax'], self.world_bounds['ymin'])
        self.minimap.create_polygon(p1[0],p1[1], p2[0],p2[1], p3[0],p3[1], p4[0],p4[1], outline='gray', fill='lightgray', width=1)
        
        # Desenha objetos
        for obj in self.objects:
            color = obj['color']
            if obj['type'] == 'ponto':
                mx,my = world_to_minimap(*obj['points'][0])
                self.minimap.create_oval(mx-2, my-2, mx+2, my+2, fill=color)
            else:
                pts = [world_to_minimap(*p) for p in obj['points']]
                flat_pts = [c for p in pts for c in p]
                if len(flat_pts) >= 4:
                    if obj['type'] == 'poligono':
                        self.minimap.create_line(*flat_pts, flat_pts[0], flat_pts[1], width=1, fill=color)
                    else:
                        self.minimap.create_line(*flat_pts, width=1, fill=color)


        # Desenha window rotacionada
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']
        cx = (wminx + wmaxx) / 2
        cy = (wminy + wmaxy) / 2
        w = wmaxx - wminx
        h = wmaxy - wminy
        
        angle_rad = math.radians(self.rotation_angle)
        cos_a = math.cos(angle_rad)
        sin_a = math.sin(angle_rad)

        corners = [(-w/2, -h/2), (w/2, -h/2), (w/2, h/2), (-w/2, h/2)]
        rotated_corners_world = []
        for x, y in corners:
            rx = x * cos_a - y * sin_a
            ry = x * sin_a + y * cos_a
            rotated_corners_world.append((cx + rx, cy + ry))
        
        minimap_corners = [world_to_minimap(*p) for p in rotated_corners_world]
        flat = [coord for p in minimap_corners for coord in p]
        if flat:
            self.minimap.create_polygon(flat, outline='red', fill='', width=2)
    
    # Mover a window e recarregar a viewport
    def _mover_e_recarregar(self, dx, dy):
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']

        window_width = wmaxx - wminx
        window_height = wmaxy - wminy

        world_width = self.world_bounds['xmax'] - self.world_bounds['xmin']
        world_height = self.world_bounds['ymax'] - self.world_bounds['ymin']

        if window_width >= world_width or window_height >= world_height:
            center_x = (self.world_bounds['xmin'] + self.world_bounds['xmax']) / 2
            center_y = (self.world_bounds['ymin'] + self.world_bounds['ymax']) / 2
            
            new_wminx = center_x - window_width / 2
            new_wminy = center_y - window_height / 2
            new_wmaxx = center_x + window_width / 2
            new_wmaxy = center_y + window_height / 2
            
            self.window['wmin'] = (new_wminx, new_wminy)
            self.window['wmax'] = (new_wmaxx, new_wmaxy)
        else:
            movimento = 0.2 
            local_step_x = window_width * movimento * dx
            local_step_y = window_height * movimento * dy
            
            # Rotaciona o vetor de movimento conforme o ângulo da window
            angle_rad = math.radians(self.rotation_angle)
            cos_a = math.cos(angle_rad)
            sin_a = math.sin(angle_rad)
            
            step_x = local_step_x * cos_a - local_step_y * sin_a
            step_y = local_step_x * sin_a + local_step_y * cos_a

            # Verifica limites do mundo
            if wmaxx + step_x > self.world_bounds['xmax']:
                step_x = self.world_bounds['xmax'] - wmaxx
            if wminx + step_x < self.world_bounds['xmin']:
                step_x = self.world_bounds['xmin'] - wminx
            if wmaxy + step_y > self.world_bounds['ymax']:
                step_y = self.world_bounds['ymax'] - wmaxy
            if wminy + step_y < self.world_bounds['ymin']:
                step_y = self.world_bounds['ymin'] - wminy
        
            new_wminx = wminx + step_x
            new_wminy = wminy + step_y
            new_wmaxx = new_wminx + window_width
            new_wmaxy = new_wminy + window_height
        
            self.window['wmin'] = (new_wminx, new_wminy)
            self.window['wmax'] = (new_wmaxx, new_wmaxy)


        self.recalcular_matriz()
        self.desenhar_viewport()
        self.desenhar_minimapa()
        self.atualizar_displaylist()

    # Zoom in/out
    def _zoom(self, fator):
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']
        width = wmaxx - wminx
        height = wmaxy - wminy
        new_width = width * fator
        new_height = height * fator
        world_width = self.world_bounds['xmax'] - self.world_bounds['xmin']
        world_height = self.world_bounds['ymax'] - self.world_bounds['ymin']
        
        # Limita zoom out 
        if new_width > world_width:
            new_width = world_width
        if new_height > world_height:
            new_height = world_height
        
        # Limita zoom in
        MIN_SIZE = 1e-1  # Tamanho mínimo
        MAX_ZOOM_IN = 0.01  # Limite máximo de 1% do mundo
        
        min_allowed_width = world_width * MAX_ZOOM_IN
        min_allowed_height = world_height * MAX_ZOOM_IN
        
        if new_width < min_allowed_width:
            new_width = min_allowed_width
        if new_height < min_allowed_height:
            new_height = min_allowed_height
        
        if new_width < MIN_SIZE or new_height < MIN_SIZE:
            return
        
        center_x = (wminx + wmaxx) / 2
        center_y = (wminy + wmaxy) / 2
        
        new_wminx = center_x - new_width / 2
        new_wminy = center_y - new_height / 2
        new_wmaxx = center_x + new_width / 2
        new_wmaxy = center_y + new_height / 2
        
        if new_wmaxx > self.world_bounds['xmax']:
            deslocamento = self.world_bounds['xmax'] - new_wmaxx
            new_wmaxx += deslocamento
            new_wminx += deslocamento
        elif new_wminx < self.world_bounds['xmin']:
            deslocamento = self.world_bounds['xmin'] - new_wminx
            new_wminx += deslocamento
            new_wmaxx += deslocamento
        
        if new_wmaxy > self.world_bounds['ymax']:
            deslocamento = self.world_bounds['ymax'] - new_wmaxy
            new_wmaxy += deslocamento
            new_wminy += deslocamento
        elif new_wminy < self.world_bounds['ymin']:
            deslocamento = self.world_bounds['ymin'] - new_wminy
            new_wminy += deslocamento
            new_wmaxy += deslocamento
            
        self.window['wmin'] = (new_wminx, new_wminy)
        self.window['wmax'] = (new_wmaxx, new_wmaxy)
        
        self.recalcular_matriz()
        self.desenhar_viewport()
        self.desenhar_minimapa()
        self.atualizar_displaylist()


    def _indent_xml(self, elem, level=0):
        i = "\n" + level * "\t"
        if len(elem):
            if not elem.text or not elem.text.strip():
                elem.text = i + "\t"
            if not elem.tail or not elem.tail.strip():
                elem.tail = i
            for child in elem:
                self._indent_xml(child, level + 1)
            if not child.tail or not child.tail.strip():
                child.tail = i
        else:
            if level and (not elem.tail or not elem.tail.strip()):
                elem.tail = i

    # gerar XML de saída com dados originais e transformados
    def gerar_xml_saida(self):
        if not self.caminho_arquivo:
            messagebox.showwarning("Aviso", "Nenhum arquivo foi carregado ainda!")
            return
        
        try:
            caminho_saida = filedialog.asksaveasfilename(
                defaultextension=".xml",
                filetypes=[("XML files", "*.xml"), ("All files", "*.*")],
                title="Salvar XML de Saída"
            )
            if not caminho_saida:
                return
            tree = ET.parse(self.caminho_arquivo)
            root = tree.getroot()
            
            comentario = ET.Comment(" Dados transformados para coordenadas da viewport ")
            root.append(comentario)
            
            for obj in self.objects:
                if not obj['visivel']:
                    continue
                    
                if obj['type'] == 'ponto':
                    ponto_ppc = obj['clipped_points'][0]
                    x, y = self.ppc_to_viewport(ponto_ppc)
                    ponto_elem = ET.SubElement(root, 'ponto')
                    ponto_elem.set('x', f"{x:.6f}")
                    ponto_elem.set('y', f"{y:.6f}")
                    ponto_elem.set('coords', 'viewport')
                    
                elif obj['type'] == 'reta':
                    reta_elem = ET.SubElement(root, 'reta')
                    reta_elem.set('coords', 'viewport')
                    for ponto_ppc in obj['clipped_points']:
                        x, y = self.ppc_to_viewport(ponto_ppc)
                        ponto_elem = ET.SubElement(reta_elem, 'ponto')
                        ponto_elem.set('x', f"{x:.6f}")
                        ponto_elem.set('y', f"{y:.6f}")
                        
                elif obj['type'] == 'poligono':
                    for poligono_ppc in obj['clipped_polygons']:
                        poligono_elem = ET.SubElement(root, 'poligono')
                        poligono_elem.set('coords', 'viewport')
                        for ponto_ppc in poligono_ppc:
                            x, y = self.ppc_to_viewport(ponto_ppc)
                            ponto_elem = ET.SubElement(poligono_elem, 'ponto')
                            ponto_elem.set('x', f"{x:.6f}")
                            ponto_elem.set('y', f"{y:.6f}")
                        
            self._indent_xml(root)
            tree.write(caminho_saida, encoding='utf-8', xml_declaration=True)
            messagebox.showinfo("Sucesso", f"XML de saída gerado com sucesso!\nSalvo em: {caminho_saida}")
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao gerar XML de saída:\n{e}")


    # Cria ou atualiza a janela DisplayList com visualização em tempo real
    def mostrar_displaylist(self):
        if self.displaylist_window is None or not tk.Toplevel.winfo_exists(self.displaylist_window):
            # Cria nova janela
            self.displaylist_window = tk.Toplevel(self.root)
            self.displaylist_window.title("DisplayList")
            self.displaylist_window.geometry("1350x400")
            
            # Frame para a tabela com scrollbar
            table_frame = tk.Frame(self.displaylist_window)
            table_frame.pack(fill='both', expand=True, padx=5, pady=5)
            
            # Scrollbar vertical
            scrollbar_y = tk.Scrollbar(table_frame, orient='vertical')
            scrollbar_y.pack(side='right', fill='y')
            
            # Scrollbar horizontal
            scrollbar_x = tk.Scrollbar(table_frame, orient='horizontal')
            scrollbar_x.pack(side='bottom', fill='x')
            
            # Treeview para a tabela
            columns = ('ID', 'Tipo', 'Cor', 'Visível', 'Coord. Mundo', 'Coord. PPC', 'Coord. Viewport')
            self.tree = ttk.Treeview(table_frame, columns=columns, show='headings', yscrollcommand=scrollbar_y.set, xscrollcommand=scrollbar_x.set)
            
            scrollbar_y.config(command=self.tree.yview)
            scrollbar_x.config(command=self.tree.xview)
            
            # Configurar colunas
            self.tree.heading('ID', text='ID')
            self.tree.heading('Tipo', text='Tipo')
            self.tree.heading('Cor', text='Cor')
            self.tree.heading('Visível', text='Visível')
            self.tree.heading('Coord. Mundo', text='Coord. Mundo')
            self.tree.heading('Coord. PPC', text='Coord. PPC')
            self.tree.heading('Coord. Viewport', text='Coord. Viewport')
            
            self.tree.column('ID', width=40, anchor='center')
            self.tree.column('Tipo', width=80, anchor='center')
            self.tree.column('Cor', width=100, anchor='center')
            self.tree.column('Visível', width=60, anchor='center')
            self.tree.column('Coord. Mundo', width=300, anchor='w')
            self.tree.column('Coord. PPC', width=300, anchor='w')
            self.tree.column('Coord. Viewport', width=300, anchor='w')
            
            # Estilo baseado na VISIBILIDADE - cinza para invisível, preto para visível
            self.tree.tag_configure('visible', background='white', foreground='black')
            self.tree.tag_configure('invisible', background='white', foreground='gray')
            
            self.tree.pack(fill='both', expand=True)
            
            # Bind para fechamento da janela
            self.displaylist_window.protocol("WM_DELETE_WINDOW", self._fechar_displaylist)
        
        # Atualiza o conteúdo
        self.atualizar_displaylist()
    
    # Fecha a janela DisplayList
    def _fechar_displaylist(self):
        if self.displaylist_window:
            self.displaylist_window.destroy()
            self.displaylist_window = None
            
    # Atualiza os dados da DisplayList em tempo real
    def atualizar_displaylist(self):
        if self.displaylist_window is None or not tk.Toplevel.winfo_exists(self.displaylist_window):
            return
        
        # Limpa a tabela
        for item in self.tree.get_children():
            self.tree.delete(item)
        
        # Preenche a tabela com dados atualizados
        for idx, obj in enumerate(self.objects, start=1):
            tipo = obj['type']
            cor = obj['color']
            visivel = "Sim" if obj['visivel'] else "Não"
            
            # Formata coordenadas do mundo
            coord_mundo = self._formatar_coordenadas(obj['points'])
            
            # Formata coordenadas PPC
            coord_ppc = self._formatar_coordenadas(obj.get('points_ppc', []))
            
            # Formata coordenadas Viewport
            if obj['visivel']:
                if tipo == 'ponto':
                    pts_vp = [self.ppc_to_viewport(obj['clipped_points'][0])]
                elif tipo == 'reta':
                    pts_vp = [self.ppc_to_viewport(p) for p in obj['clipped_points']]
                elif tipo == 'poligono':
                    pts_vp = []
                    for poligono_ppc in obj['clipped_polygons']:
                        pts_vp.extend([self.ppc_to_viewport(p) for p in poligono_ppc])
                coord_viewport = self._formatar_coordenadas(pts_vp)
            else:
                coord_viewport = "[]"
            
            # Determina a tag
            vis_tag = 'visible' if obj['visivel'] else 'invisible'
            
            # Insere na tabela
            self.tree.insert('', 'end', 
                           values=(idx, tipo, cor, visivel, coord_mundo, coord_ppc, coord_viewport),
                           tags=(vis_tag,))
    
    # Formata lista de pontos para exibição
    def _formatar_coordenadas(self, pontos):
        if not pontos:
            return "[]"
        
        formatted = []
        for p in pontos:
            if isinstance(p, (tuple, list)) and len(p) == 2:
                formatted.append(f"({p[0]:.2f},{p[1]:.2f})")
            else:
                formatted.append(str(p))
        
        result = "[" + ", ".join(formatted) + "]"
        return result

    def mostrar_controles_do_teclado(self):
        texto_controle = """
        Controles do Teclado ---------------------------------
        
        Movimentação da Window:
        - Teclas ← → ↑ ↓ : Move na direção correspondente.
        
        Zoom (10% por click):
        - Tecla '+' ou '=' : Aproxima (Zoom In).
        - Tecla '-' : Afasta (Zoom Out).
        
        Rotação da Window:
        - Tecla '>' ou '.' : Rotaciona para a direita.
        - Tecla '<' ou ',' : Rotaciona para a esquerda.
        
        Algoritmos Implementados:
        - Linhas: Cohen-Sutherland ou Liang-Barsky (selecionável)
        - Polígonos: Weiler-Atherton
        
        Pipeline de Transformação:
        Mundo → PPC → Clipping → Viewport → Desenho
        
        DisplayList:
        - Abre janela com visualização em tempo real do pipeline
        - Atualiza automaticamente a cada transformação
        """
        messagebox.showinfo("Controles do Teclado", texto_controle)

if __name__ == "__main__":
    root = tk.Tk()
    app = Visualizador(root)
    root.mainloop()
